In [1]:
import os

from exa_py import Exa
from openai import OpenAI
import json

EXA_API_KEY= os.environ.get("EXA_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

exa = Exa(api_key = EXA_API_KEY)
client = OpenAI(api_key = OPENAI_API_KEY)
model = 'gpt-4-turbo'

In [2]:
example_topic = 'The latest news on medical uses of pychedelics.'

def generate_report_without_exa(topic):
  content = f"Write a comprehensive and professional three paragraph research report about {topic}. Include citations with source, month, and year."
  completion = client.chat.completions.create(
      model = model,
      messages=[
          {"role": "user", "content": content},
      ],
      temperature = 0
      )
  return completion.choices[0].message.content


report = generate_report_without_exa(example_topic)
report

'As of March 2023, the field of psychedelic research has continued to expand, particularly in the context of mental health treatment. Recent studies have focused on substances such as psilocybin, the psychoactive component in magic mushrooms, and MDMA, commonly known as ecstasy, for their potential therapeutic benefits. A landmark study published in the "Journal of Psychopharmacology" (January 2023) demonstrated that psilocybin therapy could significantly reduce symptoms in patients with major depressive disorder (MDD). The randomized controlled trial indicated that psilocybin, when administered in a controlled, therapeutic setting, led to rapid and sustained symptom reduction, outperforming traditional antidepressants in some cases (Davis et al., 2023).\n\nFurthermore, MDMA-assisted therapy has shown promise in treating post-traumatic stress disorder (PTSD). According to research published in "Nature Medicine" (February 2023), MDMA, when used in conjunction with psychotherapy, helped 

In [3]:
example_topic = "ADAPTIMMUNE Therapeutics plc"

def generate_report_without_exa(topic):
  content = f"Write a comprehensive and professional three paragraph research report about {topic}. Include citations with source, month, and year."
  completion = client.chat.completions.create(
      model = model,
      messages=[
          {"role": "user", "content": content},
      ],
      temperature = 0
      )
  return completion.choices[0].message.content


report = generate_report_without_exa(example_topic)
report

"**ADAPTIMMUNE Therapeutics plc Research Report**\n\n**Introduction to ADAPTIMMUNE Therapeutics plc**\nADAPTIMMUNE Therapeutics plc, a biopharmaceutical company, is at the forefront of developing novel cancer immunotherapy products based on its proprietary SPEAR (Specific Peptide Enhanced Affinity Receptor) T-cell platform. The company, headquartered in Abingdon, the UK, focuses on leveraging the body's own immune system to fight tumor cells. ADAPTIMMUNE designs, develops, and commercializes immunotherapeutic products primarily for patients with solid tumors. As of September 2023, the company's leading product candidates include afamitresgene autoleucel (formerly known as ADP-A2M4), which targets multiple solid tumor types, including sarcoma, melanoma, and ovarian cancer (ADAPTIMMUNE, 2023).\n\n**Clinical Development and Strategic Collaborations**\nADAPTIMMUNE has been actively advancing its pipeline with multiple ongoing clinical trials. The most advanced therapy, afamitresgene autole

In [5]:
# Exa searches each subtopic
def exa_search_each_subquery(subqueries):
  list_of_query_exa_pairs = []
  for query in subqueries:
    search_response = exa.search_and_contents(
      query,
      num_results=5,
      use_autoprompt=True,
      start_published_date="2023-06-01", # To give us only recent information post-June 2023
      highlights={"num_sentences": 5},
    )
    query_object = {
        'subquery': query,
        'results': search_response.results
    }
    list_of_query_exa_pairs.append(query_object)
  return list_of_query_exa_pairs



In [7]:
# Reformat Exa results into a string
def format_exa_results_for_llm(list_of_query_exa_pairs, content_slice=750):
    formatted_string = ""
    for i in list_of_query_exa_pairs:
      formatted_string += f"[{i['subquery']}]:\n"
      for result in i['results']:
        content = result.text if result.text else " ".join(result.highlights)
        publish_date = result.published_date
        formatted_string += f"URL: {result.url}\nContent: {content}\nPublish Date: {publish_date}\n"
      formatted_string += "\n"

    return formatted_string

#example
#print(format_exa_results_for_llm(example_list_of_query_exa_pairs))

In [8]:
def generate_report_from_exa_results(topic, list_of_query_exa_pairs):
  formatted_exa_content = format_exa_results_for_llm(list_of_query_exa_pairs)
  content = f"Write a comprehensive and professional three paragraph research report about {topic} based on the provided information. Include citations in the text using footnote notation ([citation #]), for example [2]. First provide the report, followed by a single `References` section that only lists the URLs (and their published date) used, in the format [#] <url>. For the published date, only include the month and year. Reset the citations index and ignore the order of citations in the provided information. Here is the information: {formatted_exa_content}."

  completion = client.chat.completions.create(
      model=model,
      messages=[
          {"role": "user", "content": content},
      ],
  )

  return completion.choices[0].message.content

In [10]:
def create_custom_function(num_subqueries):
    properties = {}
    for i in range(1, num_subqueries + 1):
        key = f'subquery_{i}'
        properties[key] = {
            'type': 'string',
            'description': 'Search queries that would be useful for generating a report on my main topic'
        }

    custom_function = {
        'name': 'generate_exa_search_queries',
        'description': 'Generates Exa search queries to investigate the main topic',
        'parameters': {
            'type': 'object',
            'properties': properties
        }
    }

    return [custom_function]

In [9]:
def generate_subqueries_from_topic(topic, num_subqueries=6):
    content =  f"I'm going to give you a topic I want to research. I want you to generate {num_subqueries} interesting, diverse search queries that would be useful for generating a report on my main topic. Here is the main topic: {topic}."
    custom_functions = create_custom_function(num_subqueries)
    completion = client.chat.completions.create(
        model = model,
        messages=[
            {"role": "user", "content": content},
        ],
        temperature=0,
        functions = custom_functions,
        function_call = 'auto',
    )

    json_response = json.loads(completion.choices[0].message.function_call.arguments)
    return list(json_response.values())


In [11]:
topic = 'The latest about ADAPTIMMUNE Therapeutics plc'

def generate_report(topic):
  subqueries = generate_subqueries_from_topic(topic)
  list_of_query_exa_pairs = exa_search_each_subquery(subqueries)
  report = generate_report_from_exa_results(topic, list_of_query_exa_pairs)
  return report

example_report = generate_report(topic)
print(example_report)

Adaptimmune Therapeutics plc has recently highlighted significant progress in its clinical trials and product development, particularly in the area of T-cell therapy. The company announced the U.S. FDA Acceptance of its Biologics License Application for Afami-cel, aimed at treating advanced Synovial Sarcoma with Priority Review. If approved, Afami-cel will be the first engineered T-cell therapy for solid tumors and marks a potential breakthrough offering for patients suffering from this type of cancer. This advancement follows the successful completion of the submission of its rolling Biologics License Application backed by encouraging data from their pivotal trial SPEARHEAD-1, which met its primary endpoint for efficacy [1][2].

Financially, Adaptimmune has demonstrated robust performance. As of the end of 2023, the company reported significant year-over-year revenue growth. Earnings for the year ended December 31, 2023, were notably higher at $60.3 million compared to $27.1 million i

In [12]:
topic = 'The latest about Wavepaths, a company focused on developing generative music to adjoin psychedelic-assisted therapies.'

def generate_report(topic):
  subqueries = generate_subqueries_from_topic(topic)
  list_of_query_exa_pairs = exa_search_each_subquery(subqueries)
  report = generate_report_from_exa_results(topic, list_of_query_exa_pairs)
  return report

example_report = generate_report(topic)
print(example_report)

Wavepaths has demonstrated significant advancements in the integration of generative music within psychedelic-assisted therapies, emphasizing how continuous and variable-length sounds can enhance the therapeutic experience. The development of cutting-edge Deep Generative Models (DGMs) which struggle with expressing continuous sounds, especially those mimicking natural movements like "driving" or the resonance from a bow on a string, is a priority for the company as they strive to make their music platform more expressive and adaptive to therapeutic needs [1]. Additionally, leveraging high-quality audio tokenization techniques, as utilized by the Descript Audio Codec (DAC), has allowed Wavepaths to preserve sound fidelity at lower sampling costs, ultimately enhancing the digital music experience in therapeutic settings [2].

In the sphere of healthcare partnerships, Wavepaths has aligned with entities committed to enhancing health sector operations and experiences, such as Wavemaker 360